In [223]:
import bs4, requests, json
import pandas as pd

# List of all contests (gym = true)

In [224]:
url = "http://codeforces.com/api/contest.list?gym=true"

res = requests.get(url)

soup = bs4.BeautifulSoup(res.text,'html.parser')
newDictionary=json.loads(str(soup))

all_contests_df = pd.DataFrame(newDictionary['result'])

all_contests_df.head()


,id,name,type,phase,frozen,durationSeconds,description,difficulty,kind,season,preparedBy,icpcRegion,country,city,startTimeSeconds,relativeTimeSeconds,websiteUrl
0,100001,2010 Codeforces Beta Round #1 (training),ICPC,FINISHED,False,7200,This is the only contest for testing Codeforce...,3.0,Training Contest,2010-2011,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,100002,2002-2003 ACM-ICPC Northeastern European Regio...,ICPC,FINISHED,False,18000,NaN,4.0,Official ICPC Contest,2002-2003,MikeMirzayanov,Northeastern Europe Region,Russia,Saint Petersburg,NaN,NaN,NaN
2,100003,2008-2009 Всероссийская командная олимпиада шк...,ICPC,FINISHED,False,18000,NaN,3.0,Official School Contest,2008-2009,Edvard,NaN,Russia,Saint Petersburg,1.453514e+09,186897661.0,NaN
3,100004,Local Contest,ICPC,FINISHED,False,10800,NaN,3.0,Official School Contest,2017-2018,MikeMirzayanov,NaN,Uzbekistan,Namangan,1.511928e+09,128484061.0,NaN
4,100005,2010-2011 Цикл интернет-олимпиад. Вторая коман...,ICPC,FINISHED,False,18000,NaN,3.0,Training Contest,2010-2011,PavelKunyavskiy,NaN,Russia,NaN,NaN,NaN,NaN


In [225]:
all_contests_df.kind.value_counts()

Official ICPC Contest                         303
Official School Contest                       284
Training Contest                              244
School/University/City/Region Championship    200
Training Camp Contest                         126
Official International Personal Contest       113
Opencup Contest                                21
Name: kind, dtype: int64

In [240]:
opencup_contests_df = all_contests_df[all_contests_df.kind == 'Opencup Contest']
opencup_contests_df.to_csv('Data/contests/opencup_contests.csv', index=False)
opencup_contests_df.head()

,id,name,type,phase,frozen,durationSeconds,description,difficulty,kind,season,preparedBy,icpcRegion,country,city,startTimeSeconds,relativeTimeSeconds,websiteUrl
317,100514,Crypto Cup 1.0,ICPC,FINISHED,False,23400,Cryptography Open CUP,3.0,Opencup Contest,2014-2015,Damon,NaN,Iran,Tehran,1.414845e+09,225567061.0,http://eprint.iacr.org
345,100571,Hello 2015 (Div.2),ICPC,FINISHED,False,10800,NaN,3.0,Opencup Contest,2014-2015,PrinceOfPersia,NaN,Iran,Tehran,1.420645e+09,219766861.0,NaN
374,100633,"2015 ICL, Finals, Div. 1",ICPC,FINISHED,False,18000,NaN,4.0,Opencup Contest,2014-2015,Scalar,NaN,Russia,Kazan’,NaN,NaN,http://icl.ru/turnir/
375,100637,"2015 ICL, Finals, Div. 2",ICPC,FINISHED,False,18000,NaN,3.0,Opencup Contest,2014-2015,Scalar,NaN,Russia,Kazan’,NaN,NaN,http://icl.ru/turnir/
492,100917,"2015-2016 XVI Open Cup, Grand Prix of Bashkort...",ICPC,FINISHED,False,18000,NaN,4.0,Opencup Contest,2015-2016,MikeMirzayanov,NaN,Russia,Izhevsk,1.457210e+09,183202261.0,http://opencup.ru/


In [227]:
opencup_contests_ids = list(opencup_contests_df.id)

# Final standing of open cup contests

In [229]:
def clean_rows(df):
    # data = pd.DataFrame
    df['contestId'] = df.party.apply(lambda x: x['contestId'])
    df['members'] = df.party.apply(lambda x: x['members'] if x['members'] else [])
    df['teamName'] = df.party.apply(lambda x: x['teamName'] if ('teamName' in x.keys()) else '')
    df['participantType'] = df.party.apply(lambda x: x['participantType'] if ('participantType' in x.keys()) else '')
    df['ghost'] = df.party.apply(lambda x: x['ghost'] if ('ghost' in x.keys()) else '')
    return df




In [235]:
final_df = pd.DataFrame()
standing_dfs = []
problems_dfs = []
for i in opencup_contests_ids:
    url = "http://codeforces.com/api/contest.standings?contestId={}&from=1&count=150&showUnofficial=true".format(i)
    res = requests.get(url)
    while(res.status_code != 200):
        res = requests.get(url)
    soup = bs4.BeautifulSoup(res.text,'html.parser')
    newDictionary=json.loads(str(soup))
    standing_df = pd.DataFrame(newDictionary['result']['rows'])
    standing_dfs.append((clean_rows(standing_df)))
    problems_dfs.append(pd.DataFrame(newDictionary['result']['problems']))

In [238]:
standing_df = pd.concat(standing_dfs)
standing_df.to_csv('Data/contests/standing_opencup.csv', index=False)

In [239]:
probems_df = pd.concat(problems_dfs)
probems_df.to_csv('Data/contests/problems_opencup.csv', index=False)